<a href="https://colab.research.google.com/github/JPxYang/PythonLearn/blob/main/%E6%8E%A8%E8%96%A6%E6%BC%94%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
credentials = service_account.Credentials.from_service_account_file('service-project-data-prod-ffbd7f3273b8.json')


def query_bigquery_table(project_id):
  # 初始化 BigQuery 客戶端
  bq_client = bigquery.Client(credentials=credentials)
  # 構建查詢
  query = f"""
        with postreads as (
              select * from `WellW.post_reads`
        ),
        posts as (
              select * from `WellW.posts`
        ),
        d7 as (
            SELECT  COUNT(pr.read_log_id) AS count ,pr.post_id as article_seq ,pr.post_id as id , p.title
            FROM postreads as pr
            JOIN posts as p  ON pr.post_id = p.id
            WHERE DATE(pr.create_at) >= DATE_ADD( CURRENT_DATE('+08'), INTERVAL -1 DAY)
            GROUP BY pr.post_id, p.title
            ORDER BY count DESC
        ),
        d7x as (
            SELECT *   ,row_number() over (order by count desc) as row_num
            from d7
        ),
        d8 as (
            SELECT  COUNT(pr.read_log_id) AS count ,pr.post_id as article_seq ,pr.post_id as id , p.title
            FROM postreads as pr
            JOIN posts as p  ON pr.post_id = p.id
            WHERE DATE(pr.create_at) >= DATE_ADD( CURRENT_DATE('+08'), INTERVAL -2 DAY)
            and  DATE(pr.create_at) <= DATE_ADD( CURRENT_DATE('+08'), INTERVAL -1 DAY)
            GROUP BY pr.post_id, p.title
            ORDER BY count DESC
        ),
        d8x as (
            SELECT *   ,row_number() over (order by count desc) as row_num
            from d8
        ),
        postlikeCounts as (
            SELECT   count(post_id) as postlikeCount ,post_id
            from `WellW.post_likes`
            group by post_id
        ),
        final as(
        select d7x.id,d7x.title ,d7x.row_num as row7,ifnull(d8x.row_num,99) as row8
        ,posts.create_at
        ,plc.postlikeCount
        from d7x left join d8x on d7x.id=d8x.id
        join posts on d7x.id=posts.id
        join postlikeCounts as plc on plc.post_id = d7x.id
        )
        select * from final
       """

  # 執行查詢
  query_job = bq_client.query(query , project=project_id )
  # 取得查詢結果
  results = query_job.result()

  return results

In [3]:
results = query_bigquery_table('t-solstice-405908')
# 將結果轉換為 JSON 格式
data_list = []
for row in results:
  data_list.append(dict(row.items()))

In [ ]:
data_list[0]

In [5]:
len(data_list)

74

In [ ]:
y = json.loads( python_dict['doc_entities'] )
for x in y["entities"]:
  if( x["name"] != 'NUMBER' ):
    print( x )

In [ ]:
moderCategoriesJosn = json.loads( python_dict['doc_moderationCategories'] )
for categoriesX in moderCategoriesJosn["categories"]:
  print( categoriesX )

{'name': 'Toxic', 'confidence': 0.02596125565469265}
{'name': 'Insult', 'confidence': 0.017338726669549942}
{'name': 'Profanity', 'confidence': 0.009878856129944324}
{'name': 'Derogatory', 'confidence': 0.004792527295649052}
{'name': 'Sexual', 'confidence': 0.0024352301843464375}
{'name': 'Death, Harm & Tragedy', 'confidence': 0.001607717014849186}
{'name': 'Violent', 'confidence': 0.0}
{'name': 'Firearms & Weapons', 'confidence': 0.0}
{'name': 'Public Safety', 'confidence': 0.0014393667224794626}
{'name': 'Health', 'confidence': 0.09630458801984787}
{'name': 'Religion & Belief', 'confidence': 0.002452483167871833}
{'name': 'Illicit Drugs', 'confidence': 0.002336448524147272}
{'name': 'War & Conflict', 'confidence': 0.0}
{'name': 'Politics', 'confidence': 0.0}
{'name': 'Finance', 'confidence': 0.014897579327225685}
{'name': 'Legal', 'confidence': 0.0}


In [19]:
# Rewritten code from /r2/r2/lib/db/_sorts.pyx

from datetime import datetime, timedelta
from math import log

epoch = datetime(1970, 1, 1)

def epoch_seconds(date):
    td = date - epoch
    return td.days * 86400 + td.seconds + (float(td.microseconds) / 1000000)

def score(ups, downs):
    return ups - downs

def hot(ups, downs, date):
    s = score(ups, downs)
    order = log(max(abs(s), 1), 2)
    sign = 1 if s > 0 else -1 if s < 0 else 0
    seconds = epoch_seconds(date) - 1134028003
    return round(sign * order + seconds / 45000, 7)

In [ ]:
a1 = hot(1,1,datetime(2024, 1, 25) )
a2 = hot(29,8,datetime(2024, 1, 30) )
a1,a2

(12713.6177111, 12724.5399304)

In [ ]:
import pandas as pd
Postdata_list = []
for PostData in data_list:
  print( hot(PostData["postlikeCount"] , 0 ,PostData["create_at"] ) ,PostData["id"] ,PostData["title"])
  listX = []
  listX.append(hot(PostData["postlikeCount"] , 0 ,PostData["create_at"] ))
  listX.append(PostData["id"])
  listX.append(PostData["title"])
  listX.append(PostData["create_at"])
  listX.append(PostData["postlikeCount"])
  Postdata_list.append(listX)

In [25]:
def takeSecond(elem):
  return elem[0]

In [26]:
Postdata_list.sort(key=takeSecond, reverse=True)
Postdata_list

[[12728.7626173,
  '01HNF0J998DQPM0VXM7HGGFCRZ',
  '誰說運動一定要去健身房？過年就把遊戲籌碼(錢)換成運動組數吧！',
  datetime.datetime(2024, 1, 31, 5, 41, 14, 152048),
  9],
 [12728.1760159,
  '01HNF0G18HC9FHAXXPF0N14GC1',
  '過年大掃除嗎？',
  datetime.datetime(2024, 1, 31, 5, 40, 0, 401167),
  6],
 [12728.0401938,
  '01HNF5YQMY5Y1JBHXPCQN0FRFF',
  '分享你過年最討厭聽到長輩說的那些話！',
  datetime.datetime(2024, 1, 31, 7, 15, 24, 958683),
  5],
 [12728.0179579,
  '01HNESPZBJ7GEBV6PJDD4A49J9',
  '#WellW活動  發文按讚留言就能拿獎金？',
  datetime.datetime(2024, 1, 31, 3, 41, 27, 794548),
  6],
 [12727.8863036,
  '01HNEZBCWR2R69BB3PXHAWZTXQ',
  '過年去哪玩?',
  datetime.datetime(2024, 1, 31, 5, 19, 59, 896682),
  5],
 [12727.8824158,
  '01HNEZ621JYQ9P3F965NFCWRRF',
  '請假提前返鄉好嗎?',
  datetime.datetime(2024, 1, 31, 5, 17, 4, 946721),
  5],
 [12727.8721985,
  '01HNEYR11GCNZR6ZFHNSFHM0AF',
  '有人除夕輪流在娘家、婆家過的嗎?',
  datetime.datetime(2024, 1, 31, 5, 9, 25, 168530),
  5],
 [12727.753386,
  '01HNF7EZ0WX90K0K4W8RVFN0M9',
  '2023幸福企業出爐啦~~',
  datetime.datetime(2024, 1, 